In [36]:
import matplotlib
matplotlib.use('TkAgg')  # or try 'Qt5Agg' if TkAgg doesn't work
import numpy as np
from PIL import Image
import os

In [37]:
# Print current working directory
print(f"Current working directory: {os.getcwd()}")

# Specify the image path
image_path = 'vai_gt1.tif'
print(f"Attempting to open image: {image_path}")

# Check if file exists
if not os.path.exists(image_path):
    print(f"Error: The file {image_path} does not exist.")
    exit()

Current working directory: d:\Kerja\Test Gabor
Attempting to open image: vai_gt1.tif


In [38]:
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox, RectangleSelector
from matplotlib.patches import Rectangle

# Specify the image paths
original_image_path = 'vai1.tif'
ground_truth_path = 'vai_gt1.tif'  # Adjust this to your ground truth image filename

# Check if files exist
if not os.path.exists(original_image_path):
    print(f"Error: The file {original_image_path} does not exist.")
    exit()
if not os.path.exists(ground_truth_path):
    print(f"Error: The file {ground_truth_path} does not exist.")
    exit()

In [ ]:
# Load the images
original_img = Image.open(original_image_path)
ground_truth_img = Image.open(ground_truth_path)
original_array = np.array(original_img)
ground_truth_array = np.array(ground_truth_img)

# Create figure and axes
fig, (ax_original, ax_ground_truth, ax_text) = plt.subplots(1, 3, figsize=(18, 6), 
                                                            gridspec_kw={'width_ratios': [3, 3, 2]})

# Display the images
im_original = ax_original.imshow(original_array)
im_ground_truth = ax_ground_truth.imshow(ground_truth_array)

# Remove axis ticks from the images
ax_original.set_xticks([])
ax_original.set_yticks([])
ax_ground_truth.set_xticks([])
ax_ground_truth.set_yticks([])

# Set titles for the images
ax_original.set_title("Original Image")
ax_ground_truth.set_title("Ground Truth")

# Create text for displaying information
info_text = ax_text.text(0.05, 0.95, '', transform=ax_text.transAxes, 
                         verticalalignment='top', wrap=True)
ax_text.axis('off')

# Global variables
square_sizes = [5, 15, 20, 25, 40, 50, 100]  # List of square sizes
current_size_index = 0
current_square_size = square_sizes[current_size_index]

# Create rectangle patches for both images
rect_original = Rectangle((0, 0), current_square_size, current_square_size, 
                          fill=False, edgecolor='red', linewidth=2)
rect_ground_truth = Rectangle((0, 0), current_square_size, current_square_size, 
                              fill=False, edgecolor='red', linewidth=2)
ax_original.add_patch(rect_original)
ax_ground_truth.add_patch(rect_ground_truth)

def update_info(event):
    if event.inaxes in [ax_original, ax_ground_truth]:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < original_array.shape[1] and 0 <= y < original_array.shape[0]:
            original_value = original_array[y, x]
            ground_truth_value = ground_truth_array[y, x]
            info = f'Mouse position: ({x}, {y})\n'
            info += f'Original value: {original_value}\n'
            info += f'Ground truth value: {ground_truth_value}'
        else:
            info = f'Mouse position: ({x}, {y})\nOut of bounds'
        
        x1, y1 = x, y
        x2, y2 = x + current_square_size, y + current_square_size
        selected_area = f'Selected area: ({x1},{y1}) to ({x2},{y2})'
        info += f'\n{selected_area}\nCurrent square size: {current_square_size}x{current_square_size}'
        
        info_text.set_text(info)
        
        # Update rectangle positions
        rect_original.set_xy((x, y))
        rect_ground_truth.set_xy((x, y))
        
        fig.canvas.draw_idle()

# Connect the event to the figure
fig.canvas.mpl_connect('motion_notify_event', update_info)

def toggle_selector(event):
    global current_size_index, current_square_size
    if event.key == 'n':
        current_size_index = (current_size_index + 1) % len(square_sizes)
        current_square_size = square_sizes[current_size_index]
        print(f"Square size set to: {current_square_size}x{current_square_size}")
        rect_original.set_width(current_square_size)
        rect_original.set_height(current_square_size)
        rect_ground_truth.set_width(current_square_size)
        rect_ground_truth.set_height(current_square_size)
        update_info(event)

fig.canvas.mpl_connect('key_press_event', toggle_selector)

# Enable zooming
ax_original.set_title('Original Image\nUse scroll wheel to zoom\nPress "n" to change square size')
ax_ground_truth.set_title('Ground Truth\nUse scroll wheel to zoom\nPress "n" to change square size')

def zoom(event):
    if event.inaxes in [ax_original, ax_ground_truth]:
        cur_xlim = event.inaxes.get_xlim()
        cur_ylim = event.inaxes.get_ylim()
        
        xdata = event.xdata
        ydata = event.ydata
        if event.button == 'up':
            scale_factor = 0.9
        elif event.button == 'down':
            scale_factor = 1.1
        else:
            scale_factor = 1
        
        event.inaxes.set_xlim([xdata - (xdata - cur_xlim[0]) * scale_factor,
                               xdata + (cur_xlim[1] - xdata) * scale_factor])
        event.inaxes.set_ylim([ydata - (ydata - cur_ylim[0]) * scale_factor,
                               ydata + (cur_ylim[1] - ydata) * scale_factor])
        fig.canvas.draw()

fig.canvas.mpl_connect('scroll_event', zoom)

plt.tight_layout()
plt.show()

Square size set to: 15x15
Square size set to: 26x26
Square size set to: 50x50
Square size set to: 100x100
Square size set to: 5x5
Square size set to: 15x15
Square size set to: 26x26
Square size set to: 50x50
